In [1]:
import pickle
import os
from perses.annihilation.rest import RESTTopologyFactory
from perses.annihilation.lambda_protocol import RESTState
from openmmtools.states import SamplerState, ThermodynamicState, CompoundThermodynamicState
from openmmtools import cache, utils
from perses.dispersed.utils import configure_platform
cache.global_context_cache.platform = configure_platform(utils.get_fastest_platform().getName())
from simtk import openmm, unit
import math
from openmmtools.constants import kB
from openmmtools import mcmc, multistate
import argparse
import copy
from perses.dispersed import feptasks
import mdtraj as md
import numpy as np
from perses.app.relative_point_mutation_setup import PointMutationExecutor

# Set up logger
import logging
_logger = logging.getLogger()
_logger.setLevel(logging.INFO)


INFO:numexpr.utils:Note: NumExpr detected 48 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.


conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA


INFO:rdkit:Enabling RDKit 2020.09.4 jupyter extensions


In [2]:
phase = 'complex'
name = 'asn'
state = 0
length = 1
move_length = 1
timestep = 4
radius = 0.7

In [3]:
# Load rhtf 
dir = "/data/chodera/zhangi/perses_benchmark/neq/14/36/"
i = os.path.basename(os.path.dirname(dir))
path = os.path.join(dir, f"{i}_{phase}_{state}.pickle")
_logger.info(f"path: {path}")
htf = pickle.load(open(path, "rb" ))



INFO:root:path: /data/chodera/zhangi/perses_benchmark/neq/14/36/36_complex_0.pickle


In [4]:
# Build REST factory
_logger.info("Generating REST factory")
_logger.info(f"radius:{radius} nm")
# query_indices = [atom.index for atom in list(htf.hybrid_topology.residues)[int(args.resid)].atoms]
for res in htf.hybrid_topology.residues:
    if res.resSeq == 501 and res.chain.index == 0:
        mutated_res = res
query_indices = [atom.index for atom in mutated_res.atoms]
_logger.info(f"query indices {query_indices}")
traj = md.Trajectory(np.array(htf.hybrid_positions), htf.hybrid_topology)
solute_atoms = list(traj.topology.select("is_protein"))
rest_atoms = list(md.compute_neighbors(traj, radius, query_indices, haystack_indices=solute_atoms)[0])

INFO:root:Generating REST factory
INFO:root:radius:0.7 nm
INFO:root:query indices [2605, 2606, 2607, 2608, 2609, 2610, 2611, 2612, 2613, 2614, 2615, 2616, 2617, 2618, 183578, 183579, 183580, 183581, 183582, 183583, 183584, 183585, 183586, 183587, 183588, 183589, 183590, 183591, 183592]
<ipython-input-4-49abc7cdb52b>:12: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  rest_atoms = list(md.compute_neighbors(traj, radius, query_indices, haystack_indices=solute_atoms)[0])


In [5]:
for atom in htf.hybrid_topology.atoms:
    if atom.index in rest_atoms:
        print(atom.index, atom)

1090 ARG403-HA
1094 ARG403-CG
1095 ARG403-HG2
1096 ARG403-HG3
1097 ARG403-CD
1098 ARG403-HD2
1112 GLY404-H
1618 ASN437-HD21
1636 ASN439-HA
1637 ASN439-CB
1638 ASN439-HB2
1640 ASN439-CG
1641 ASN439-OD1
1642 ASN439-ND2
1643 ASN439-HD21
1644 ASN439-HD22
1691 ASP442-O
1692 SER443-N
1694 SER443-CA
1695 SER443-HA
1696 SER443-CB
1697 SER443-HB2
1698 SER443-HB3
1699 SER443-OG
1700 SER443-HG
1701 SER443-C
1703 LYS444-N
1704 LYS444-H
1723 LYS444-C
1724 LYS444-O
1728 VAL445-HA
1746 GLY446-C
1747 GLY446-O
1748 GLY447-N
1749 GLY447-H
1750 GLY447-CA
1751 GLY447-HA2
1752 GLY447-HA3
1753 GLY447-C
1755 ASN448-N
1756 ASN448-H
1757 ASN448-CA
1759 ASN448-CB
1760 ASN448-HB2
1761 ASN448-HB3
1767 ASN448-C
1768 ASN448-O
1779 TYR449-CE1
1780 TYR449-HE1
1781 TYR449-CZ
1782 TYR449-OH
1783 TYR449-HH
1784 TYR449-CE2
1785 TYR449-HE2
2518 TYR495-HB3
2531 TYR495-C
2532 TYR495-O
2533 GLY496-N
2534 GLY496-H
2535 GLY496-CA
2536 GLY496-HA2
2537 GLY496-HA3
2538 GLY496-C
2539 GLY496-O
2540 PHE497-N
2541 PHE497-H
2542 PHE49

In [6]:
len(rest_atoms)

367

In [7]:
phase = 'complex'
name = 'tyr'
state = 1
length = 1
move_length = 1
timestep = 4
radius = 0.5

In [4]:
# Load rhtf 
dir = "/data/chodera/zhangi/perses_benchmark/neq/14/83/"
i = os.path.basename(os.path.dirname(dir))
path = os.path.join(dir, f"{i}_{phase}_{state}.pickle")
_logger.info(f"path: {path}")
htf = pickle.load(open(path, "rb" ))



INFO:root:path: /data/chodera/zhangi/perses_benchmark/neq/14/83/83_complex_1.pickle


In [8]:
# Build REST factory
_logger.info("Generating REST factory")
_logger.info(f"radius:{radius} nm")
# query_indices = [atom.index for atom in list(htf.hybrid_topology.residues)[int(args.resid)].atoms]
for res in htf.hybrid_topology.residues:
    if res.resSeq == 501 and res.chain.index == 0:
        mutated_res = res
query_indices = [atom.index for atom in mutated_res.atoms]
_logger.info(f"query indices {query_indices}")
traj = md.Trajectory(np.array(htf.hybrid_positions), htf.hybrid_topology)
solute_atoms = list(traj.topology.select("is_protein"))
rest_atoms = list(md.compute_neighbors(traj, radius, query_indices, haystack_indices=solute_atoms)[0])

INFO:root:Generating REST factory
INFO:root:radius:0.5 nm
INFO:root:query indices [2605, 2606, 2607, 2608, 2609, 2610, 2611, 2612, 2613, 2614, 183493, 183494, 183495, 183496, 183497, 183498, 183499, 183500, 183501, 183502, 183503, 183504, 183505, 183506, 183507]
<ipython-input-8-49abc7cdb52b>:12: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  rest_atoms = list(md.compute_neighbors(traj, radius, query_indices, haystack_indices=solute_atoms)[0])


In [9]:
for atom in htf.hybrid_topology.atoms:
    if atom.index in rest_atoms:
        print(atom.index, atom)

1641 ASN439-OD1
1643 ASN439-HD21
1696 SER443-CB
1697 SER443-HB2
1698 SER443-HB3
1700 SER443-HG
1751 GLY447-HA2
1756 ASN448-H
1783 TYR449-HH
2533 GLY496-N
2534 GLY496-H
2535 GLY496-CA
2536 GLY496-HA2
2537 GLY496-HA3
2538 GLY496-C
2539 GLY496-O
2540 PHE497-N
2541 PHE497-H
2542 PHE497-CA
2543 PHE497-HA
2544 PHE497-CB
2545 PHE497-HB2
2546 PHE497-HB3
2547 PHE497-CG
2548 PHE497-CD1
2549 PHE497-HD1
2558 PHE497-C
2559 PHE497-O
2560 GLN498-N
2561 GLN498-H
2562 GLN498-CA
2563 GLN498-HA
2564 GLN498-CB
2565 GLN498-HB2
2566 GLN498-HB3
2567 GLN498-CG
2568 GLN498-HG2
2569 GLN498-HG3
2570 GLN498-CD
2571 GLN498-OE1
2572 GLN498-NE2
2573 GLN498-HE21
2574 GLN498-HE22
2575 GLN498-C
2576 GLN498-O
2577 PRO499-N
2587 PRO499-CA
2588 PRO499-HA
2589 PRO499-C
2590 PRO499-O
2591 THR500-N
2592 THR500-H
2593 THR500-CA
2594 THR500-HA
2595 THR500-CB
2596 THR500-HB
2597 THR500-CG2
2598 THR500-HG21
2601 THR500-OG1
2602 THR500-HG1
2603 THR500-C
2604 THR500-O
2605 ALA501-N
2606 ALA501-H
2607 ALA501-CA
2608 ALA501-HA
2609 

In [14]:
len(rest_atoms)

55